<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Практика" data-toc-modified-id="Практика-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Практика</a></span><ul class="toc-item"><li><span><a href="#Загрузка-DataFrame" data-toc-modified-id="Загрузка-DataFrame-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка DataFrame</a></span><ul class="toc-item"><li><span><a href="#Задача-1" data-toc-modified-id="Задача-1-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Задача 1</a></span></li></ul></li><li><span><a href="#Теория-вероятности.-События" data-toc-modified-id="Теория-вероятности.-События-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Теория вероятности. События</a></span><ul class="toc-item"><li><span><a href="#Задача-2" data-toc-modified-id="Задача-2-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Задача 2</a></span></li><li><span><a href="#Задача-3" data-toc-modified-id="Задача-3-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Задача 3</a></span></li></ul></li></ul></li></ul></div>

# Практика
Используемые библиотеки

In [10]:
import pandas as pd
import numpy as np
import requests
import io
import re
import itertools
import random


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
"""url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))"""
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
df['Вывод'] = [re.sub('[^-,0-9]','', x) for x in df['Норматив']]
df['Вывод'] = [re.sub(',', '.', x) for x in df['Вывод']]
df['Результат анализа'] = [re.sub('[^,.0-9]','0', x) for x in df['Результат анализа']]
df['Результат анализа'] = pd.to_numeric(df['Результат анализа'])

In [4]:
s =[]
for i in df['Вывод']:
    s.append(i.split('-'))
df['Вывод']=s
df.head()

,Показатель,Единица измерений,Результат анализа,Норматив,Вывод
0,pH,единицы pH,8.40,в пределах 6-9,"[6, 9]"
1,Запах,баллы,1.00,не более 2-3,"[2, 3]"
2,Цветность,градусы,0.00,не более 30,[30]
3,Жёсткость,мг-эквл/дм3,9.20,в пределах 7-10,"[7, 10]"
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",[1.5]


In [5]:
for i in range(len(df)):
    if df.iloc[i]['Норматив'].find('в пределах ') >= 0:
        if float(df.iloc[i]['Вывод'][0]) <=df.iloc[i]['Результат анализа'] <= float(df.iloc[i]['Вывод'][1]):
            df.iloc[i, df.columns.get_loc('Вывод')] = "Норма"
        else: 
            df.iloc[i, df.columns.get_loc('Вывод')] = "Не Норма"
    elif df.iloc[i]['Норматив'].find('не более ') >= 0:
        if len(df.iloc[i]['Вывод']) > 1 and df.iloc[i]['Результат анализа'] <= float(df.iloc[i]['Вывод'][1]):
            df.iloc[i, df.columns.get_loc('Вывод')] = "Норма"
        elif len(df.iloc[i]['Вывод']) == 1 and df.iloc[i]['Результат анализа'] <= float(df.iloc[i]['Вывод'][0]):
            df.iloc[i, df.columns.get_loc('Вывод')] = "Норма"
        else: 
            df.iloc[i, df.columns.get_loc('Вывод')] = "Не Норма"

In [9]:
df = df.set_index(["Показатель"])
df

,Единица измерений,Результат анализа,Норматив,Вывод
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,Норма
Запах,баллы,1.000,не более 2-3,Норма
Цветность,градусы,0.000,не более 30,Норма
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",Норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Норма
Нитраты (по NO3),мг/дм3,24.000,не более 45,Норма
Фосфаты (P),мг/дм3,0.360,"не более 3,5",Норма
Хлориды (Cl),мг/дм3,200.000,не более 350,Норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [75]:
chance = [10, 100, 1000, 10000, 100000, 1000000, 10000000]

for k in chance:  
    score = 0
    
    for q in range(k):  
        busket = ['я','я','я','я','a','a','a','a','a']
        count = 3
        positive = 0
        
        for m in range(count):
            index = random.randint(0, len(busket)-1)
            if busket[index] == 'a':
                positive += 1
            busket.pop(index)
            
        if positive == 3:
            score += 1
            
    print(f"Вероятность выбрать три апельсина равна {round(score / k, 3)}, за {k} попыток")


Вероятность выбрать три апельсина равна 0.2, за 10 попыток
Вероятность выбрать три апельсина равна 0.1, за 100 попыток
Вероятность выбрать три апельсина равна 0.119, за 1000 попыток
Вероятность выбрать три апельсина равна 0.119, за 10000 попыток
Вероятность выбрать три апельсина равна 0.118, за 100000 попыток
Вероятность выбрать три апельсина равна 0.119, за 1000000 попыток
Вероятность выбрать три апельсина равна 0.119, за 10000000 попыток


<div class="alert alert-block alert-success">
Эксперементально ответ подтверждается при больших количествах попыток
</div>

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [76]:
chance = [10, 100, 1000, 10000, 100000, 1000000, 10000000]

for k in chance:  
    score = 0
    
    for q in range(k):
        details = ['нестд','нестд','нестд','стд','стд','стд','стд','стд','стд','стд']
        index = random.randint(0, len(details)-1)
        
        if details[index] == 'нестд':
            details.pop(index)
            index = random.randint(0, len(details)-1)
            
            if details[index] == 'стд':
                score  += 1
                
    print(f"Вероятность проверить ровно две детали {score/k}, за {k} попыток")


Вероятность проверить ровно две детали 0.1, за 10 попыток
Вероятность проверить ровно две детали 0.22, за 100 попыток
Вероятность проверить ровно две детали 0.253, за 1000 попыток
Вероятность проверить ровно две детали 0.2312, за 10000 попыток
Вероятность проверить ровно две детали 0.23317, за 100000 попыток
Вероятность проверить ровно две детали 0.234248, за 1000000 попыток
Вероятность проверить ровно две детали 0.2333791, за 10000000 попыток


<div class="alert alert-block alert-success">
Эксперементально ответ подтверждается при больших количествах попыток
</div>